#### 1.1.2 _Programming Exercise:_ Verify that inverse nonce values `k` and `-k` generate inverse points `R` and `-R`

In [ ]:
# Generate a random scalar and its associated curve point
k, R = generate_key_pair()

# Find the x- and y-coordinates from R
# Use the get_x() and get_y() methods
R_x = R.get_x()
R_y = R.get_y()

print("R_x: {}".format(R_x))
print("R_y: {}\n".format(R_y))

# Find k's inverse (SECP256K1_ORDER - k)
# Extract the secret value from k using .secret
minus_k = SECP256K1_ORDER - k.secret

# Generate the key pair from minus_k using generate_key_pair() function with minus_k as an argument
minus_k_key, minus_R = generate_key_pair(minus_k)

# Find the x- and y-coordinates from -R
minus_R_x = minus_R.get_x()
minus_R_y = minus_R.get_y()

print("minus_R_x: {}".format(minus_R_x))
print("minus_R_y: {}\n".format(minus_R_y))

assert R_x == minus_R_x
assert SECP256K1_FIELD_SIZE - R_y == minus_R_y

print("Success!")

#### 1.1.3 _Programming Exercise:_ Sign a message with Schnorr

In [ ]:
msg = sha256(b'message')

# Generate a private/public key pair
d, P = generate_key_pair()

# Generate a nonce scalar and associated nonce point
k, R = generate_key_pair()

# Check that nonce is quadratic residue modulo the field size
# If not, negate k
# Method: jacobi_symbol(R.get_y(), SECP256K1_FIELD_SIZE)
if jacobi_symbol(R.get_y(), SECP256K1_FIELD_SIZE) != 1:
    k.negate()
# Note that there is no need to negate R, since we only use the x value of R below

# Generate s = k + sha256(R_x|P|msg) * d
# Method: sha256(bytes) will give you the byte digest of the SHA256 of some bytes
# Turn that digest into a ECKey object called h, and then set s = k + h * d
R_x_bytes = R.get_x().to_bytes(32, 'big')
P_bytes = P.get_bytes()
h_bytes = sha256(R_x_bytes + P_bytes + msg)
h = ECKey().set(h_bytes)
s = k + h * d

print("R: {}".format(R))
print("s: {}\n".format(s.get_bytes().hex()))

# Generate sig = R_x|s
# Method: get the x bytes from R and concatenate with the secret bytes from s
sig = R.get_x().to_bytes(32, 'big') + s.secret.to_bytes(32,'big')

print("Signature: {}\n".format(sig.hex()))

# Verify the signature
assert P.verify_schnorr(sig, msg)

print("Success!")

#### 1.1.4 _Programming Exercise:_ Signing schnorr with deterministic nonce

In [ ]:
msg = sha256(b'message')
d, P = generate_key_pair()
print("message = {}".format(msg.hex()))
print("pubkey = {}\n".format(P.get_bytes().hex()))

# Generate the nonce value k deterministically and get the nonce point R
# Method: use sha256(bytes) on the private key and message
k_bytes = sha256(d.get_bytes() + msg)
k, R = generate_key_pair(k_bytes)

# Check that nonce is quadratic residue modulo the field size
# If not, negate k
# Method: jacobi_symbol(int(y(R)), SECP256K1_FIELD_SIZE)
if jacobi_symbol(R.get_y(), SECP256K1_FIELD_SIZE) != 1:
    k.negate()

print("nonce: {}".format(k))
print("nonce point: {}\n".format(R))

# Generate s = k + sha256(R_x|P|msg) * d
# Method: sha256(bytes) will give you the byte digest of the SHA256 of some bytes
# Turn that digest into a ECKey object called h, and then set s = k + h * d
R_x_bytes = R.get_x().to_bytes(32, 'big')
P_bytes = P.get_bytes()
h_bytes = sha256(R_x_bytes + P_bytes + msg)
h = ECKey().set(h_bytes)
s = k + h * d

print("R: {}".format(R))
print("s: {}\n".format(s.get_bytes()))

# Generate sig = R_x|s
# Method: get the x bytes from R and concatenate with the secret bytes from s
sig = R.get_x().to_bytes(32, 'big') + s.secret.to_bytes(32,'big')

print("Signature: {}\n".format(sig.hex()))

# Generate a signature using the ECKey.sign_schnorr(msg) method
# This generates the nonce deterministically, so should return the same signature
sig2 = d.sign_schnorr(msg)

# Verify and compare signature(s)
assert P.verify_schnorr(sig, msg)
assert P.verify_schnorr(sig2, msg)
assert sig == sig2

print("Success!")